In [6]:
import os

In [7]:
%pwd

'c:\\Users\\HP\\Desktop\\DS\\Projects\\13.wine_quality_prediction_project\\research'

In [8]:
os.chdir("../")

In [9]:
%pwd

'c:\\Users\\HP\\Desktop\\DS\\Projects\\13.wine_quality_prediction_project'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metrics_file_name: str
    target_column: str

In [14]:
from mlProject.utils.common import read_yaml,create_directories
from mlProject.constants import *
from mlProject.logging import logger

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
            ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        module_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path= config.model_path,
            metrics_file_name= config.metrics_file_name,
            target_column= schema.name
        )
        return module_evaluation_config


In [17]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
import numpy as np
import pandas as pd
import joblib
from mlProject.utils.common import save_json

class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config
    
    def evaluation_metrics(self,actual,predict):
        r2=r2_score(actual,predict)
        mse= mean_squared_error(actual,predict)
        rmse =np.sqrt(mse)
        mae=mean_absolute_error(actual,predict)
        return r2,rmse,mae

    def save_evluation(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test =test_data.drop([self.config.target_column],axis=1)
        y_test = test_data[[self.config.target_column]]

        prediction = model.predict(X_test)

        (r2,rmse,mae) = self.evaluation_metrics(y_test,prediction)

        #saving to local
        scores = {"r2 score": r2, "rmse score": rmse, "mae score": mae}
        save_json(path = Path(self.config.metrics_file_name), data=scores)




    



In [18]:
#pipeline
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config =model_evaluation_config)
    model_evaluation.save_evluation()
except Exception as e:
    raise e

[2024-07-06 17:06:05,646: INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-07-06 17:06:05,649: INFO:common:yaml file: params.yaml loaded successfully]
[2024-07-06 17:06:05,654: INFO:common:yaml file: schema.yaml loaded successfully]
[2024-07-06 17:06:05,658: INFO:common:created directory at: artifacts]
[2024-07-06 17:06:05,658: INFO:common:created directory at: artifacts/model_evaluation]
[2024-07-06 17:06:05,856: INFO:common:json file saved at: artifacts\model_evaluation\metrics.json]
